# 既存Deltaテーブルからのストリーミングデータの生成

このノートブックでは、波形データを保持する既存のDelta Lakeテーブルからストリーミングデータセットを生成します。このためには、サンプルを一定間隔で到着するものとして取り扱います。患者が到着した際には、それぞれの観察期間における患者に対応するレコードを生成します。

**注意**
このノートブックは明示的に停止しない限り、処理を継続します。

**参考資料**
- [Delta Lake、Keras、MLflowを用いた機械学習による医療機器データのモニタリング \- Qiita](https://qiita.com/taka_yayoi/items/65e463a3eab84d4e2ce7)
- [Monitoring patient medical device data with ML \+ Delta Lake, Keras, and MLflow](https://databricks.com/blog/2019/09/12/monitor-medical-device-data-with-machine-learning-using-delta-lake-keras-and-mlflow-on-demand-webinar-and-faqs-now-available.html)

<table>
  <tr><th>作者</th><th>Databricks Japan</th></tr>
  <tr><td>日付</td><td>2021/7/9</td></tr>
  <tr><td>バージョン</td><td>1.0</td></tr>
  <tr><td>クラスター</td><td>8.3ML</td></tr>
</table>
<img style="margin-top:25px;" src="https://jixjiadatabricks.blob.core.windows.net/images/databricks-logo-small-new.png" width="140">

In [0]:
# 以下のパスを前のノートブックと揃えてください
LOCAL_DATA_PATH = '/tmp/takaaki.yayoi@databricks.com/hls/ptb-diagnostic-ecg'
DELTA_PATH = '/tmp/takaaki.yayoi@databricks.com/hls/ecg/staged/'

In [0]:
import random
import time
from pyspark.sql.functions import lit

base_table = sqlContext.read.format("delta").load(DELTA_PATH)
records = base_table.count()

# 適宜変更してください
stream_path = '/tmp/takaaki.yayoi@databricks.com/hls/ecg/streaming/'

dbutils.fs.rm(stream_path, True)

ts = 0

base_table.limit(1).withColumn('time_interval', lit(0)).write.format('delta').save(stream_path)

for i in range(records - 1):
  derived_table = base_table.limit(i)
  
  sleep_time = random.randint(1, 5)
  
  for j in range(sleep_time):
    ts += 1
    
    base_table.limit(i).withColumn('time_interval', lit(ts)).write.format('delta').mode('append').save(stream_path)
    
while True:
  
  ts += 1
  base_table.withColumn('time_interval', lit(ts)).write.format('delta').mode('append').save(stream_path)

# END